In [1]:
import numpy as np
import random as rd
from scipy.stats import chi2
from math import log, sqrt

EJERCICIO 1

In [2]:
frec_obs=np.array([120,114,92,85,34,33,45,11,5],dtype=float)
p=np.array([0.22,0.20,0.19,0.12,0.09,0.08,0.07,0.02,0.01], dtype=float)
n=int(np.sum(frec_obs))
frec=n*p

In [3]:
#Estadístico
def T(n,frec,p):
    k=len(frec)
    t=0
    for i in range(k):
        t+=((frec[i]-(n*p[i]))**2) / (n*p[i])
    return t

In [4]:
#Genero una v.a con la distribucion global
def gen():
    u=rd.random()
    if u < 0.22:
        return 0
    if u < 0.42:
        return 1 
    if u < 0.61:
        return 2
    if u < 0.73:
        return 3
    if u < 0.82:
        return 4
    if u < 0.9:
        return 5
    if u < 0.97:
        return 6
    if u < 0.99:
        return 7
    else:
        return 8

In [5]:
#Retorna la frecuencia de cada color de una muestra de la 
#variable global de n datos.
def gen_muestra(n):
    muestra=np.zeros(9, dtype=float)
    for i in range(n):
        muestra[gen()]+=1
    return muestra


In [6]:
def est_p_valor(n, frec, p,it):
    k=len(frec)
    t_est=0
    t=T(n, frec, p)
    for i in range(it):
        #Simulo muestra
        frec_sim=gen_muestra(n)  #frecuencia de n datos con distr F
    
        #Calculo un nuevo estadistico
        t1=T(n,frec_sim, p) 
        if t1>=t:
            t_est+=1
    return t_est/it

In [7]:
p_value=est_p_valor(n, frec_obs, p,10000)
t=T(n,frec_obs,p)

In [ ]:
print("Estimacion del p-valor con aproximacion Chi2: p-valor={}".format(1 - chi2.cdf(t,len(frec_obs)-1)))
print("Estimacion del p-valor con 10000 simulaciones: p-valor={}".format(p_value))

In [ ]:
print("En ambos casos rechazamos la hipótesis nula")

EJERCICIO 2

In [10]:
def fun(t):
    return (t-3)**2

In [11]:
def Poisson_adelgazamiento_mejorado(T, interv, lam, fun):
    j = 0 
    t = -log (1 - rd.random()) / lam[j]
    NT = 0
    Eventos = []
    while t <= T:
        if t <= interv[j]:
            V = rd.random()
            if V < fun(t) / lam[j]:
                NT += 1
                Eventos.append(t)
            t += -log(1 - rd.random()) / lam[j]
        else:
            t = interv[j] + (t - interv[j]) * lam[j] / lam[j + 1]
            j += 1
    return NT, Eventos

In [ ]:
t_eventos=Poisson_adelgazamiento_mejorado(6, [1,2,4,5,6], [9,4,1,4,9], fun)[1]
print("Lista de los tiempos de eventos:\n{}".format(t_eventos))

EJERCICIO 4

b)

In [13]:
#Generaremos puntos en el primer cuadrante 
#con cota x<=2, y<=1

def point(): #Retorna 1 si esta dentro de la elipse, 0 en caso contrario
    x=np.random.uniform(0,2)
    y=rd.random()
    if ((x/2)**2) + (y**2) <=1:
        return 1
    else:
        return 0

In [14]:
def area_rec(it):
    punto=0
    for i in range(it):
        punto+=point()
    return 8*punto/it 
#*8 debido al area del rectangulo

In [ ]:
area_rec(10000)
print("El area estimada es: {}".format(area_rec(10000)))

c)

In [16]:
#Para IC de 95% tenemos que z_alfa_2= 1.96
#tol es la maxima longitud que le permito al IC
#Si L=2*a_alfa_2*S /sqrt(n) 
# => it_necesarias=(2*dev*z_alfa_2/tol)^2
def est_area(tol,z_alfa_2,max_it):
    n=1
    mean_n=point()
    dev=0
    ancho=0

    while n<max_it and (n<=100 or ancho>=tol):
        #Genero un nuevo dato
        dato=point()
        
        #Actualizo la media
        mean_n1= mean_n + ((dato-mean_n)/(n+1))
        
        #Actualizo la desviacion
        dev= (1-(1/n))*(dev**2) + (n+1)*((mean_n1 - mean_n)**2)
        dev=dev**0.5

        ancho= 2*z_alfa_2*dev/sqrt(n)
        mean_n=mean_n1
        n+=1
    
    return (8*mean_n - ancho, 8*mean_n + ancho), int((2*dev*z_alfa_2/tol)**2), n-1
#*8 debido al area del rectangulo

In [ ]:
IC, it_t, it=est_area(0.1,1.96,10000)
print("El IC es:[{},{}]".format(IC[0], IC[1]))
print("Las simulaciones teoricas necesarias: {}".format(it_t))
print("Las simulaciones necesarias en este caso: {}".format(it))